In [248]:
import pandas as pd 
import numpy as np 
from sqlalchemy import create_engine 
import os
from dotenv import load_dotenv
from urllib.parse import quote_plus
from sklearn.utils import resample
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline  import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import cloudpickle
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier

In [249]:
from sklearn import set_config
set_config(display='diagram')

In [250]:
db_members = pd.read_csv("tables/members_v3.csv")
db_train = pd.read_csv("tables/train_v2.csv")
db_transactions = pd.read_csv("tables/transactions_v2.csv")

### Taking a sample data of 50000 rows

### Filtering the rows of test and transaction based on the presence of foreign key 'msno' in members table

In [251]:
common_items = set(db_train['msno']).intersection(set(db_transactions["msno"]))

db_members = db_members[db_members["msno"].isin(common_items)]
db_members = db_members.head(10000)

In [252]:
db_transactions = db_transactions[db_transactions['msno'].isin(db_members['msno'])]
db_train = db_train[db_train['msno'].isin(db_members['msno'])]

In [253]:
zeros = db_train[db_train['is_churn'] == 0]
ones = db_train[db_train['is_churn'] == 1]
print(zeros.shape)
print(ones.shape)

(9354, 2)
(646, 2)


### Performing undersampling of 0's to match the number fo 1's in the is_churn column

In [254]:
# undersampling 0's to match the number of 1's
zeros_undersampled = resample(zeros,replace=False,n_samples=len(ones),random_state=42)
db_train = pd.concat([zeros_undersampled,ones])

# suffling the results
db_train = db_train.sample(frac=1,random_state=42).reset_index(drop=True)

print(ones.count())
print(zeros_undersampled.count())
print(db_train.shape)

msno        646
is_churn    646
dtype: int64
msno        646
is_churn    646
dtype: int64
(1292, 2)


Python script for changing csv to sql script

In [255]:

def generate_sql(database,output_file,table_name):
    output = output_file
    with open(output, "w", encoding="utf-8") as f:
        for index, row in database.iterrows():
            values = ",".join(
                [ f"'{str(x).replace('\'', '\\\'')}'" if pd.notna(x) else 'Null' for x in row]
            )
            f.write(f"INSERT INTO {table_name} VALUES ({values});\n")

In [256]:
generate_sql(db_members, "queries/members.sql", "members")
generate_sql(db_transactions, "queries/transactions.sql", "transactions")
generate_sql(db_train, "queries/train.sql", "train")


### Importing the main view as a database

In [257]:
load_dotenv( dotenv_path= "login.env")
username = os.getenv("mysql_username")
password = os.getenv("mysql_password")
host = os.getenv("mysql_host")
port = os.getenv("mysql_port")
database = os.getenv("mysql_database")
password = quote_plus(password)

In [258]:
engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")

### Reading the main view and importing it as database

In [259]:
main_db = pd.read_sql("select * from main_view", con=engine)
main_db.head()

,msno,city,gender,registered_via,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_churn
0,lGNzCvnLti9+BU4FMw33tx0LLlYXA5NsOg7TisczAg8=,1,None,7,41,30,149,149,1,2017-03-09,2017-04-09,0
1,V1AfXf3sneA3iPwmyYmYqoXZAcK+p/pUcSeJsAlv/xA=,9,male,9,34,30,149,149,1,2017-03-31,2017-04-30,0
2,dVbTa7zFuB4+l9ADU0Av9eIyteTVb4UzFTCu+x9uZqs=,14,male,7,41,30,129,129,1,2017-03-24,2017-04-24,0
3,A0SIokZFXJYip+25syK/RpfpbI8q0+tZVdMXNfq3w4M=,1,None,7,41,30,100,100,1,2017-03-26,2017-04-26,0
4,DyOdSQrUBa185/HAz6xJBwvpiDifbdBaQ20/k389+DM=,1,None,7,41,30,99,99,1,2017-03-19,2017-04-19,0


### Checking for null values

In [260]:
print(main_db.isna().sum())

msno                        0
city                        0
gender                    782
registered_via              0
payment_method_id           0
payment_plan_days           0
plan_list_price             0
actual_amount_paid          0
is_auto_renew               0
transaction_date            0
membership_expire_date      0
is_churn                    0
dtype: int64


In [261]:
main_db = main_db.dropna(ignore_index=True)

In [262]:
print(main_db.isna().sum())

msno                      0
city                      0
gender                    0
registered_via            0
payment_method_id         0
payment_plan_days         0
plan_list_price           0
actual_amount_paid        0
is_auto_renew             0
transaction_date          0
membership_expire_date    0
is_churn                  0
dtype: int64


In [263]:
main_db['transaction_date'] = pd.to_datetime(main_db['transaction_date'],format='%Y%m%d')
main_db['membership_expire_date'] = pd.to_datetime(main_db['membership_expire_date'], format='%Y%m%d')


### Dividing data for creating training, testing and validation dataset

In [264]:
rows, features = main_db.shape
x, y = np.hsplit(main_db,[features-1])

c:\Users\apoor\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
c:\Users\apoor\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [265]:
y = y.squeeze()
y = y.astype(int)

In [266]:
y.dtypes

dtype('int64')

### Checking if plan_list_price is same as actual_ammount_paid

In [267]:
count = 0
for i in range(0,len(x)):
    if(x.loc[i,"plan_list_price"] == x.loc[i,"actual_amount_paid"]):
        count += 1
print("Number of same values in each row :"+str(count))
print("Size of x:"+str(x.shape))

Number of same values in each row :1170
Size of x:(1189, 11)


### Dropping plan_list_price

In [268]:
x = x.drop(columns=["plan_list_price"])

In [269]:
x.head()

,msno,city,gender,registered_via,payment_method_id,payment_plan_days,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date
0,V1AfXf3sneA3iPwmyYmYqoXZAcK+p/pUcSeJsAlv/xA=,9,male,9,34,30,149,1,2017-03-31,2017-04-30
1,dVbTa7zFuB4+l9ADU0Av9eIyteTVb4UzFTCu+x9uZqs=,14,male,7,41,30,129,1,2017-03-24,2017-04-24
2,4fLQSp6469Rm1XbU3JoO5EHGOH9oyncStOEHsdTLXYs=,14,female,9,39,30,149,1,2017-02-28,2017-04-06
3,4fLQSp6469Rm1XbU3JoO5EHGOH9oyncStOEHsdTLXYs=,14,female,9,39,30,149,1,2017-03-31,2017-05-06
4,1mLIcr3KuTej13nEnSW6pDjXekZvNlsgCs71dZtI1y0=,13,male,7,41,30,149,1,2017-03-05,2017-04-05


In [270]:
x["payment_plan_days"].unique()

array([30, 100, 90, 7, 395, 195, 180, 410, 120, 360, 240, 60, 1, 365, 400,
       10, 200], dtype=object)

## Dropping msno

In [271]:
x = x.drop(columns=["msno"])

### Creating Test Train Split

In [272]:
x_train, x_test, x_valid = np.split(x,[int(0.7*len(x)-1), int(0.9*len(x))-1])
y_train, y_test, y_valid = np.split(y,[int(0.7*len(y)-1), int(0.9*len(y))-1])

c:\Users\apoor\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
c:\Users\apoor\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


## Data Transformation Pipeline

### 1. Perfroming One Hot encoding on the gender column

In [273]:
gen_encoding = ColumnTransformer([
    ("gender", OneHotEncoder(),[1])
],remainder='passthrough')

### 2. Creating a new column which gives the duration of the subscription, by creating a custom transformer

In [274]:
class durationTransform(BaseEstimator,TransformerMixin):
    def fit(self,x,y=None):
        
        return self
    
    def transform(self,x):
        
        # chacking if the input is dataframe or a numpy array
        if isinstance(x,pd.DataFrame):
            db = x.copy()
            db.head()
        else:
            db = pd.DataFrame(x,columns=["transaction_date","membership_expire_date"])
            db.head()
        
        # making sure the required input is in datetime format
        db["transaction_date"] = pd.to_datetime(db["transaction_date"])
        db["membership_expire_date"] = pd.to_datetime(db["membership_expire_date"])
        
        result = (db["membership_expire_date"] - db["transaction_date"]).dt.days
        return result.values.reshape(-1,1)
        

In [275]:
subs_time = ColumnTransformer([
    ("duration_in_days", durationTransform(), [8,9])
    ], remainder='passthrough')

## Creating the pipeline

In [276]:
pipe = Pipeline([
    ('gen_encoding', gen_encoding),
    ('subs_time', subs_time)
])
pipe

,steps,"[('gen_encoding', ...), ('subs_time', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('gender', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [277]:
pipe.fit(x_train,y_train)
result_from_pipe = pipe.transform(x_train)
x_train = pd.DataFrame(result_from_pipe, columns=["duration_of_subscription","female","male","city","registered_via","payment_method_id","payment_plan_days","actual_amount_paid","is_auto_renew"])
result_from_test = pipe.transform(x_test)
x_test = pd.DataFrame(result_from_test, columns=["duration_of_subscription","female","male","city","registered_via","payment_method_id","payment_plan_days","actual_amount_paid","is_auto_renew"])
result_from_valid = pipe.fit_transform(x_valid)
x_valid = pd.DataFrame(result_from_valid, columns=["duration_of_subscription","female","male","city","registered_via","payment_method_id","payment_plan_days","actual_amount_paid","is_auto_renew"])

## Final DataFrame

In [278]:
x_train.head()

,duration_of_subscription,female,male,city,registered_via,payment_method_id,payment_plan_days,actual_amount_paid,is_auto_renew
0,30,0.0,1.0,9,9,34,30,149,1
1,31,0.0,1.0,14,7,41,30,129,1
2,37,1.0,0.0,14,9,39,30,149,1
3,36,1.0,0.0,14,9,39,30,149,1
4,31,0.0,1.0,13,7,41,30,149,1


In [279]:
x_train.describe()

,duration_of_subscription,female,male,city,registered_via,payment_method_id,payment_plan_days,actual_amount_paid,is_auto_renew
count,831,831.0,831.0,831,831,831,831,831,831
unique,196,2.0,2.0,19,5,25,15,20,2
top,30,0.0,1.0,13,9,41,30,149,1
freq,152,449.0,449.0,213,442,174,686,453,602


## Improving the efficiency of models

## Training Models and calculating the accuracy

### 1. Logistic regression model

In [280]:
lg = LogisticRegression(max_iter=100)
lg.fit(x_train,y_train)
y_pred = lg.predict(x_test)
score = accuracy_score(y_pred,y_test)
print("Accuracy for Logistic Regression: "+str(round((score*100),2))+"%")

Accuracy for Logistic Regression: 73.53%


c:\Users\apoor\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### 2. Decision Tree model

In [281]:
dst = tree.DecisionTreeClassifier()
dst.fit(x_train,y_train)
y_pred = dst.predict(x_test)
score = accuracy_score(y_pred,y_test)
print("Accuracy for Descision Trees: "+str(round((score*100),2))+"%")

Accuracy for Descision Trees: 88.24%


### 3. Random forest model

In [282]:
rf = RandomForestClassifier(max_depth=10,random_state=1)
rf.fit(x_train,y_train)
y_pred = rf.predict(x_test)
score = accuracy_score(y_pred,y_test)
print("Accuracy for Random Forest: "+str(round((score*100),2))+"%")

Accuracy for Random Forest: 87.39%


### 4. Support Vector Machines

In [283]:
svm = SVC()
svm.fit(x_train,y_train)
y_pred = svm.predict(x_test)
score = accuracy_score(y_pred,y_test)
print("Accuracy for SVM: "+str(round((score*100),2))+"%")

Accuracy for SVM: 74.79%


### 5. K-Nearest Neighbours

In [284]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train,y_train)
y_pred = knn.predict(x_test)
score = accuracy_score(y_pred,y_test)
print("Accuracy for K-Nearest Neighbour: "+str(round((score*100),2))+"%")

Accuracy for K-Nearest Neighbour: 86.55%


## Performing Adabost and Voting classification

In [285]:
voting = VotingClassifier(
    [('rf', RandomForestClassifier(n_estimators=50,random_state=45)),
    ('svm', SVC(probability=True)),
    ('lg', LogisticRegression(max_iter=1000))], voting = 'soft')

voting.fit(x_train,y_train)
y_pred = voting.predict(x_test)
score = accuracy_score(y_pred,y_test)
print("Accuracy for Voting Classifier: "+str(round((score*100),2))+"%")

Accuracy for Voting Classifier: 82.35%


In [286]:
adabost = AdaBoostClassifier(
    rf,n_estimators=50,learning_rate=0.10,random_state=45
)
adabost.fit(x_train,y_train)
y_pred = adabost.predict(x_test)
score = accuracy_score(y_test,y_pred)
print("Accuracy for adaboost: "+str(round((score*100),2))+"%")

Accuracy for adaboost: 89.08%


## Using Pickle for saving the Random Forest Model using JobLib

In [287]:
with open("model/model.pickle","wb") as f:
 cloudpickle.dump(adabost,f)

with open("model/pipe.pickle","wb") as f:
 cloudpickle.dump(pipe,f)